In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 17 13:11:12 2023

@author: Debra Hogue

Modified RankNet by Lv et al. to use Tensorflow not Pytorch
and added additional comments to explain methods

Paper: Simultaneously Localize, Segment and Rank the Camouflaged Objects by Lv et al.
"""

import tensorflow as tf
from tensorflow.keras import activations, layers, losses
import numpy as np
import os, argparse
from datetime import datetime
from Attention.ResNet_models import Generator
from data import get_loader
from utils import adjust_lr, AvgMeter
from scipy import misc
import cv2
from data import test_dataset
from PIL import ImageFile
import matplotlib.pyplot as plt
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import tensorflow.keras.applications.resnet50 as models # instantiates the ResNet50 architecture 

from utils import l2_regularisation
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
gpus = tf.config.list_physical_devices('GPU')
try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=5632)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

tf.config.optimizer.set_experimental_options({"Memory optimizer": True, "Layout optimizer": True})
tf.keras.mixed_precision.set_global_policy('mixed_float16')

1 Physical GPUs, 1 Logical GPUs
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Quadro RTX 3000, compute capability 7.5


In [2]:

# parser = argparse.ArgumentParser()
# parser.add_argument('--epoch', type=int, default=1, help='epoch number')
# parser.add_argument('--lr_gen', type=float, default=2.5e-5, help='learning rate for generator')
# parser.add_argument('--batchsize', type=int, default=2, help='training batch size')
# parser.add_argument('--trainsize', type=int, default=480, help='training dataset size')
# parser.add_argument('--decay_rate', type=float, default=0.9, help='decay rate of learning rate')
# parser.add_argument('--decay_epoch', type=int, default=40, help='every n epochs decay learning rate')
# parser.add_argument('--feat_channel', type=int, default=32, help='reduced channel of saliency feat')
# opt = parser.parse_args()
# print('Generator Learning Rate: {}'.format(opt.lr_gen))
epoch = 1
decay_rate = 0.97
decay_epoch = 65
batchsize = 3

In [23]:

# build models
generator = Generator(channel=32)


# generator_params = generator.parameters()
# generator_optimizer = tf.optimizers.Adam(generator_params, opt.lr_gen)


image_root = 'dataset/train/Imgs/'
gt_root = 'dataset/train/GT/'
fix_root = 'dataset/train/Fix/'

# train_loader = get_loader(image_root, gt_root, fix_root,batchsize=opt.batchsize, trainsize=opt.trainsize)
# total_step = len(train_loader)

CE = losses.BinaryCrossentropy(from_logits=True)
mse_loss = losses.MeanSquaredError()
size_rates = [0.75,1,1.25]  # multi-scale training

def structure_loss(pred, mask):
    padded = tf.pad(mask,tf.constant([[0,0],[15,15],[15,15],[0,0]]))
    pooled =tf.nn.avg_pool2d(padded, ksize=31, strides=1, padding="VALID")
    weit  = 1+5*tf.abs(pooled-mask)
    weit = tf.squeeze(weit,[3])
    wbce= tf.nn.sigmoid_cross_entropy_with_logits(mask,pred)
   
    wbce= tf.math.reduce_mean(wbce)
    wbce  = tf.math.reduce_sum((weit*wbce),axis=[1,2]) /tf.reduce_sum(weit,axis=[1,2])
    mask =tf.squeeze(mask,[3])
    pred  = tf.math.sigmoid(pred)
    pred = tf.squeeze(pred,[3])
    inter = tf.math.reduce_sum((pred*mask)*weit,axis=[1,2])
    union = tf.math.reduce_sum((pred+mask)*weit, axis=[1,2])
    wiou  = 1-(inter+1)/(union-inter+1)
    return tf.math.reduce_mean(wbce+wiou)


        
             
def loss_function(y_true,y_pred):
    gts, fixs = tf.unstack(y_true,2,0)
    gts, _ = tf.split(gts, [1,2], 3)
    fixs, _ = tf.split(fixs, [1,2], 3)
    fix_pred, cod_pred1, cod_pred2 = tf.unstack(y_pred,num=3,axis=0)
    tf.summary.image("fixation",fix_pred)
    fix_loss = mse_loss(tf.keras.activations.sigmoid(fix_pred),fixs)
    cod_loss1 = structure_loss(cod_pred1, gts)
    cod_loss2 = structure_loss(cod_pred2, gts)
    fix_loss_scaler = tf.squeeze(fix_loss)
    tf.summary.scalar("fix_loss",fix_loss)
    test= fix_loss + cod_loss1 + cod_loss2
    return  test 
    
def on_epoch_end( epoch, lr):
    decay = decay_rate ** (epoch // decay_epoch)
    new_lr = lr * decay
    print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, lr, new_lr))
    return new_lr
        





In [ ]:

data = get_loader(image_root, gt_root, fix_root, 480, batchsize, size_rates)
fig = plt.figure(figsize=(10, 7))

mse_loss = losses.MeanSquaredError()
#for i in data:
 #   img, label = i
  #  gts, fixs = tf.unstack(label,2,0)
  #  gts, _ = tf.split(gts, [1,2], 3)
  #  fixs, _ = tf.split(fixs, [1,2], 3)
    
    
    #res2 = fixs
    #res2 = tf.image.resize(res2, size=tf.constant([WW,HH]), method=tf.image.ResizeMethod.BILINEAR)
    #res2 = tf.math.sigmoid(res2).numpy().squeeze()
    #es2 = (res2 - res2.min()) / (res.max() - res2.min() + 1e-8)
fix1 = cv2.imread('dataset/train/train/COD10K-CAM-1-Aquatic-13-Pipefish-634.png')   
fixGT = cv2.imread('dataset/train/Fix/COD10K-CAM-1-Aquatic-13-Pipefish-634.png') 

  

fig.add_subplot(1, 2, 1)
plt.imshow(fixGT)
fig.add_subplot(1, 2, 2)
plt.imshow(fix1)

fix1= fix1.astype(float)
fixGT=fixGT.astype(float)

fix1 = tf.constant(fix1)
fixGT = tf.constant(fixGT)
fix1=tf.math.sigmoid(fix1)
fixGT=tf.math.sigmoid(fixGT) 
diff = fix1-fixGT
print(diff.shape)
loss = tf.math.reduce_mean(tf.math.square(diff))
print(loss)
loss = mse_loss(fixGT,fix1)
print(loss)
print(fix1)
print(fixGT)
    

In [ ]:
        
if __name__ == '__main__':
    
  
    
    
    
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch='10, 15',write_images=True)
    
    op= tf.keras.optimizers.Adam(learning_rate=2.5e-5, name='Adam')
    
    generator.compile(optimizer=op, loss=loss_function, run_eagerly=False)
    
    
    data = get_loader(image_root, gt_root, fix_root, 480, batchsize, size_rates)
    print(len(data))
    generator.fit(x=data,batch_size=batchsize, epochs=epoch, verbose='auto' , callbacks=[tensorboard_callback, tf.keras.callbacks.LearningRateScheduler(on_epoch_end)])
     
     

2001

Epoch: 0. Reducing Learning Rate from 2.499999936844688e-05 to 2.499999936844688e-05
 575/2001 [=======>......................] - ETA: 39:46 - loss: 190.9509

In [ ]:

save_path = 'models/Resnet/'
    





dataset_path = 'dataset/Test/'

In [ ]:
if not os.path.exists(save_path):
        os.makedirs(save_path)
print("Generator is build")
generator.save("results/model")

In [ ]:
test_datasets = ['Mine', 'CAMO']


In [ ]:

for dataset in test_datasets:
    save_path = './results/ResNet50/' + dataset + '/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    image_root = dataset_path + dataset + '/Imgs/'
    test_loader = test_dataset(image_root, 480)

    for i in range(test_loader.size):
        print(i)
        image, HH, WW, name = test_loader.load_data()
        ans = generator(image)
        _,generator_pred, _  = tf.unstack(ans,num=3,axis=0)
        res = generator_pred
        res = tf.image.resize(res, size=tf.constant([WW,HH]), method=tf.image.ResizeMethod.BILINEAR)
        res = tf.math.sigmoid(res).numpy().squeeze()
        res = 255*(res - res.min()) / (res.max() - res.min() + 1e-8)
        print(save_path+name)
        cv2.imwrite(save_path+name, res)
        print()

In [ ]:

generator2 =  tf.keras.models.load_model('./models/FACE-100/', custom_objects={'loss_function': loss_function})

In [ ]:

for dataset in test_datasets:
    save_path = './results/small/' + dataset + '/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    image_root = dataset_path + dataset + '/Imgs/'
    test_loader = test_dataset(image_root, 480)

    for i in range(test_loader.size):
        print(i)
        image, HH, WW, name = test_loader.load_data()
        ans = generator2(image)
        generator_pred,_, _  = tf.unstack(ans,num=3,axis=0)
        print(generator_pred)
        res = generator_pred
        res = tf.image.resize(res, size=tf.constant([WW,HH]), method=tf.image.ResizeMethod.BILINEAR)
        res = tf.math.sigmoid(res).numpy().squeeze()
        res = 255*(res - res.min()) / (res.max() - res.min() + 1e-8)
        print(save_path+name)
        cv2.imwrite(save_path+name, res)
        print()